## Introduction

We here calculate the various entropy production (rates) discussed in Appendix D of the paper "Short-time Fokker-Planck propagator beyond the Gaussian approximation" (arXiv: http://arxiv.org/abs/2405.18381), which we in the following refer to as Ref. [1].

In [1]:
import sympy as sp
import helper_functions as hf

In [2]:
epsilon = sp.symbols(r'\epsilon',real=True,positive=True)

 # basic units of length, time, and diffusivity
L = sp.symbols('L',real=True,positive=True)
T = sp.symbols('T',real=True,positive=True)
D_0 = L**2 / T
D_x0 = sp.symbols(r'D(x_{0})',real=True,positive=True)

# other symbolic variables
dt = sp.symbols('\Delta{t}',real=True,nonnegative=True)
x = sp.symbols('x',real=True)
xDL = sp.symbols(r'\tilde{x}',real=True)
tDL = sp.symbols(r'\tilde{t}',real=True,nonnegative=True)
epsilon = sp.symbols(r'\epsilon',real=True,positive=True)
#

tD = sp.symbols(r'\tau_D',real=True,positive=True)

In [3]:
N_max = 13
Dk = []
for i in range(N_max):
    Dk.append(sp.symbols('D_{0}'.format(i)))
Ak = []
for i in range(N_max):
    Ak.append(sp.symbols('A_{0}'.format(i)))

In [4]:
Qk = []
for i in range(N_max):
    Qk.append(
        sp.Function('Q_{0}'.format(i))(xDL)
        )

In [5]:
D = 1 + sum([Dk[i] * epsilon**i * xDL**i for i in range(1,N_max)])
a = sum([Ak[i] * epsilon**i * xDL**i for i in range(N_max)])
P_prefac = sum([Qk[i] * epsilon**i for i in range(N_max)]) 
P0 = sp.exp(-xDL**2/2)/sp.sqrt(2*sp.pi)

## Medium entropy production rate

We now perturbatively calculate the dimensionless medium entropy production rate, as defined in Eq. (D12) of Ref. [1].

The integrand of the expression is

\begin{align}
\tilde{P}_0 \cdot ( 1 + \tilde{\epsilon} \tilde{\mathcal{Q}}_1 + \tilde{\epsilon}^2 \tilde{\mathcal{Q}}_2 + ) \cdot 
\frac{1}{\tilde{D}}
\cdot
\left[ 
    \left( \tilde{a} - \frac{1}{\tilde{\epsilon}}\partial_{\tilde{x}}\tilde{D}
    \right)^2
    + 
    \frac{\tilde{D}}{\tilde{\epsilon}}
    \partial_{\tilde{x}}
    \left( \tilde{a} - \frac{1}{\tilde{\epsilon}}\partial_{\tilde{x}}\tilde{D}
    \right)
\right].
\end{align}

We for now consider the three factors that multiply $\tilde{P}_0$ here,
and for each of the three factors calculate a power-series expansion in powers of
$\tilde{\epsilon}$. We then multply the power-series expansions an

In [6]:
def get_series_coefficients(list_of_factors,
                            N_max=3, # maximal order in epsilon we consider
                            ):
    #
    # for each of the factors, calculate the corresponding power series
    factors_series = []
    for i,factor in enumerate(list_of_factors):
        factors_series.append(sp.series(factor,epsilon,0,N_max+1).removeO())
    #
    # multiply out the power-series expansions
    series = 1
    for factor in factors_series:
        series = series * factor
    #
    #
    # from the resulting expression, determine the power series coefficients
    # at each power in epsilon.
    # Here we use that for a power series 
    #       f(x) = a_0 + a_1 * x**1 + a_2 * x**2 + ..
    # we have
    #       a_n = d^n f / dx^n |_{x = 0} / n!
    # Evaluating the derivatives seems to be faster than than using sp.series
    series_coefficients = []
    #
    # zero-th order
    current_derivative = series
    series_coefficients.append( current_derivative.subs(epsilon,0) )
    #
    # higher orders
    for i in range(1,N_max+1):
        #
        current_derivative = sp.diff(current_derivative,epsilon)/i
        series_coefficients.append( current_derivative.subs(epsilon,0) )
        #
    return series_coefficients


term0 = sp.expand((epsilon * a - sp.diff(D,xDL,1))/epsilon)
list_of_factors = [
    P_prefac ,
    sp.expand(term0**2 + D*sp.expand(sp.diff(term0,xDL,1)/epsilon)),
    1/D
    ]

dS_m_dt_integrand_prefactors = get_series_coefficients(
                                        list_of_factors=list_of_factors,
                                        N_max=8)
# dS_m_dt_integrand_prefactors[k] * P0 = integrand of integral Eq. (D12)
# at order epsilon**k

Now to evaluate the integral Eq. (D12) at each order of $\tilde{\epsilon}^k$,
we need to substitute the explicit expressions for the $\tilde{\mathcal{Q}}_k$.
We here obtain those from PySTFP, but one could also load them directly from the
output of the jupyter notebook *propagators and moments.ipynb*, which is in 
this very folder.

In [7]:
from PySTFP import PySTFP

In [8]:
def substitute_local_variables(expr,
                            remote):
    #
    expr = expr.subs(remote.L,L)
    expr = expr.subs(remote.T,T)
    expr = expr.subs(remote.t,dt)
    expr = expr.subs(remote.x,x)
    expr = expr.subs(remote.xDL,xDL)
    expr = expr.subs(remote.tDL,tDL)
    expr = expr.subs(remote.epsilon,epsilon)
    for k in range(min(len(Dk),len(remote.Dk))):
        expr = expr.subs(remote.Dk[k],Dk[k])
        expr = expr.subs(remote.Ak[k],Ak[k])
    return expr 

p = PySTFP()

Q_dict = {}
for key, value in p.Q_dict.items():
    if key == 0:
        Q_dict[key] = value
    else:
        Q_dict[key] = substitute_local_variables(expr=value,remote=p)

At each order $\tilde{\epsilon}^k$, the integrand in Eq. (D12) is a Gaussian multiplied with a polynomial. To efficiently evaluate this, we want to only calculate integrals of the form monomial times Gaussian, and want to sum over the results. 

Thus, we want to get the polynomial coefficients of the integrand in Eq. (D12).

In [9]:
def get_polynomial_coefficients(expr,variable,
                        N_max=100):
    #
    coefficients = []
    #
    current_derivative = expr
    coefficients.append(current_derivative.subs(variable,0))
    #
    for i in range(1,N_max):
        current_derivative = sp.diff(current_derivative,variable,1)/i
        coefficients.append(current_derivative.subs(variable,0))
        #
    #    
    coefficients_reversed = coefficients[::-1]
    for i,e in enumerate(coefficients_reversed):
        if e != 0:
            coefficients = (coefficients_reversed[i:])[::-1]
            break
    #
    return coefficients
    

In [10]:
def evaluate_integral(integrand_prefactor):
    # We want to evaluate the integral with integrand:
    # integrand = integrand_prefactor * P0
    global P0 # Gaussian
    #
    expr = integrand_prefactor
    #
    # substitute perturbative expressions for \tilde{\mathcal{Q}}_k
    for key, value in Q_dict.items():
        expr = expr.subs(Qk[key],value)
    expr = expr.doit()
    #
    # get polynomial coefficients
    coeffs = get_polynomial_coefficients(expr=sp.expand(expr),
                                variable=xDL)
    # coeffs[0] is coefficient of lowest power
    n_poly = len(coeffs) # highest power is n_poly - 1
    #
    # evaluate integrals for each monomial, and sum over the results
    integral = 0
    for j,coeff in enumerate(coeffs):
        #
        integral += coeff * sp.integrate( xDL**j * P0, 
                                (xDL,-sp.oo,sp.oo))
    #
    return integral


In [11]:
dS_m_dt_coefficients = {}

for i, current_prefactor in enumerate(dS_m_dt_integrand_prefactors):
    print(i+1,'of',len(dS_m_dt_integrand_prefactors),'   ',end='\r')
    #
    integral = evaluate_integral(integrand_prefactor=current_prefactor)
    #
    dS_m_dt_coefficients[i] = sp.expand(integral)

In [12]:
with open('dS_m_dt.py','w') as f:
    f.write('''
import sympy

N_max = {N_max}

Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)

dS_m_dt_dict = {{}}

'''.format(N_max = N_max)
    )
    #
    for i,current_coefficient in (dS_m_dt_coefficients).items():
        #
        if i < 0:
            if (current_coefficient != 0):
                raise RuntimeError("Medium entropy production contains "\
                        "diverging term.")
            continue
        #
        f.write('dS_m_dt_dict[{0}] = '.format(i))
        f.write(hf.sympy_expression_to_string(current_coefficient))
        f.write('\n\n\n') 
    #

## Total entropy production rate

The total entropy production rate is given by the integral in Eq. (D7) of Ref. [1]. 
The integrand is given as a product

\begin{align}
\frac{1}{\tilde{\epsilon}^2} \cdot \frac{1}{\tilde{D}} \cdot \frac{1}{\tilde{P}} \cdot (\tilde{\epsilon}^2 \tilde{j}) \cdot (\tilde{\epsilon}^2 \tilde{j})
\end{align}

with 

\begin{align}
\tilde{\epsilon^2} \tilde{j} &= \tilde{\epsilon} \tilde{a} \tilde{P} - \partial_{\tilde{x}}(\tilde{D}\tilde{P}).
\end{align}

We ignore the $\tilde{x}$-independent prefactor $1/\tilde{\epsilon}^2$ in the integrand for now (this is added back in the module PySTFP, when the total entropy production rate is evaluated using the power series coefficients we calculate here).
We then proceed as for the medium entropy production rate above.

In [13]:
P = P_prefac*P0

eps_sq_j= sp.expand((epsilon * a * P- sp.diff(D*P,xDL,1)))

list_of_factors = [1/D,
1/P,
eps_sq_j,
eps_sq_j,
]

dS_tot_dt_integrands = get_series_coefficients(
                                        list_of_factors=list_of_factors,
                                        N_max=8)

In [14]:
# we want the prefactors of the integrals, so we divide by P0
dS_tot_dt_integrand_prefactors = []

for integrand in dS_tot_dt_integrands:
    dS_tot_dt_integrand_prefactors.append ( 
         sp.expand( sp.expand(integrand) / P0 )
    )

In [15]:
# calculate the integrals at each order of epsilon**i
dS_tot_dt_coefficients = {}

for i, current_prefactor in enumerate(dS_tot_dt_integrand_prefactors):
    print(i+1,'of',len(dS_tot_dt_integrand_prefactors),'   ',end='\r')
    #
    integral = evaluate_integral(integrand_prefactor=current_prefactor)
    #
    dS_tot_dt_coefficients[i] = sp.expand(integral)

In [16]:
with open('dS_tot_dt.py','w') as f:
    f.write('''
import sympy

N_max = {N_max}

Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)

dS_tot_dt_dict = {{}}

'''.format(N_max = N_max)
    )
    #
    for i,current_coefficient in (dS_tot_dt_coefficients).items():
        #
        f.write('dS_tot_dt_dict[{0}] = '.format(i))
        f.write(hf.sympy_expression_to_string(current_coefficient))
        f.write('\n\n\n') 
    #

## Gibbs entropy

To perturbatively evaluate the Gibbs entropy, we use Eq. (D10) of Ref. [1], which reads

\begin{align}
\dot{\tilde{S}}^{\mathrm{tot}} &=
 \frac{1}{2}\ln\left[ {2 \pi} \tilde{\epsilon}^2 \right]
+\frac{1}{2}\int_{-\infty}^{\infty}dx\,\tilde{P}x^2
-\int_{-\infty}^{\infty}dx\,\tilde{P}\ln \left[ 1 
+ \tilde{\epsilon} \tilde{\mathcal{Q}}_1
+ \tilde{\epsilon}^2 \tilde{\mathcal{Q}}_2
+ ...
\right]
\end{align}

Since the first term has $\ln \tilde{\epsilon}^2$, we do not consider it for the perturbative calculation, but add it to our final result below. 

For the other terms, we proceed as for the medium entropy production rate above:


In [17]:
list_of_factors = [
    P_prefac ,
    xDL**2/2  - sp.ln(P_prefac)
    ]

S_Gibbs_integrand_prefactors = get_series_coefficients(
                                        list_of_factors=list_of_factors,
                                        N_max=8)

In [18]:
S_Gibbs_coefficients = {}

for i, current_prefactor in enumerate(S_Gibbs_integrand_prefactors):
    print(i+1,'of',len(S_Gibbs_integrand_prefactors),'   ',end='\r')
    #
    integral = evaluate_integral(integrand_prefactor=current_prefactor)
    #
    S_Gibbs_coefficients[i] = sp.expand(integral)

In [19]:
S_Gibbs_coefficients[0] += sp.log(2*sp.pi*epsilon**2)/2

In [20]:
S_Gibbs_coefficients[0]

log(2*pi*\epsilon**2)/2 + 1/2

In [21]:
with open('S_Gibbs.py','w') as f:
    f.write('''
import sympy

N_max = {N_max}

epsilon = sympy.symbols(r'\epsilon', real=True, positive=True)
Dk = sympy.symbols('D_0:%d'%N_max)
Ak = sympy.symbols('A_0:%d'%N_max)

S_Gibbs_dict = {{}}

'''.format(N_max = N_max)
    )
    #
    for i,S_coefficient in (S_Gibbs_coefficients).items():
        #
        f.write('S_Gibbs_dict[{0}] = '.format(i))
        f.write(hf.sympy_expression_to_string(S_coefficient))
        f.write('\n\n\n')
    #